# Window functions

General Elections were held in the UK in 2015 and 2017. Every citizen votes in a constituency. The candidate who gains the most votes becomes MP for that constituency.

All these results are recorded in a table ge

yr	| firstName	| lastName	| constituency	| party	| votes
---:|-----------|-----------|---------------|-------|------:
2015	| Ian	| Murray	| S14000024	| Labour	| 19293
2015	| Neil	| Hay	| S14000024	| Scottish National Party	| 16656
2015	| Miles	| Briggs	| S14000024	| Conservative | 8626
2015	| Phyl	| Meyer	| S14000024	| Green	| 2090
2015	| Pramod	| Subbaraman	| S14000024	| Liberal Democrat	| 1823
2015	| Paul	| Marshall	| S14000024	| UK Independence Party	 | 601
2015	| Colin	| Fox	| S14000024	| Scottish Socialist Party	| 197
2017	| Ian	| MURRAY	| S14000024	| Labour	| 26269
2017	| Jim	| EADIE	| S14000024	| SNP	| 10755
2017	| Stephanie Jane Harley	| SMITH	| S14000024	| Conservative	| 9428
2017	| Alan Christopher	| BEAL	| S14000024	| Liberal Democrats	| 1388

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Warming up

Show the **lastName, party** and **votes** for the **constituency** 'S14000024' in 2017.

In [2]:
ge <- dbReadTable(con, 'ge')

In [3]:
ge %>% 
    filter(constituency=='S14000024' & yr==2017) %>%
    select(lastname, party, votes)

lastname,party,votes
<chr>,<chr>,<int>
BEAL,Liberal Democrats,1388
MURRAY,Labour,26269
EADIE,SNP,10755
SMITH,Conservative,9428


## 2. Who won?

You can use the RANK function to see the order of the candidates. If you RANK using (ORDER BY votes DESC) then the candidate with the most votes has rank 1.

**Show the party and RANK for constituency S14000024 in 2017. List the output by party**

In [4]:
ge %>% 
    filter(constituency=='S14000024' & yr==2017) %>%
    mutate(rank=row_number(desc(votes))) %>%
    select(party, rank) %>% 
    arrange(party)

party,rank
<chr>,<int>
Conservative,3
Labour,1
Liberal Democrats,4
SNP,2


## 3. PARTITION BY

The 2015 election is a different PARTITION to the 2017 election. We only care about the order of votes for each year.

**Use PARTITION to show the ranking of each party in S14000021 in each year. Include yr, party, votes and ranking (the party with the most votes is 1).**

In [5]:
ge %>% 
    filter(constituency=='S14000021') %>%
    group_by(yr) %>% 
    mutate(posn=row_number(desc(votes))) %>%
    ungroup %>% 
    select(yr, party, votes, posn) %>%
    arrange(party, yr)

yr,party,votes,posn
<int>,<chr>,<int>,<int>
2015,Conservative,12465,3
2017,Conservative,21496,1
2019,Conservative,19451,2
2015,Labour,19295,2
2017,Labour,14346,2
2019,Labour,6855,3
2015,Liberal Democrats,1069,4
2017,Liberal Democrats,1112,3
2019,Liberal Democrats,4174,4


## 4. Edinburgh Constituency

Edinburgh constituencies are numbered S14000021 to S14000026.

**Use PARTITION BY constituency to show the ranking of each party in Edinburgh in 2017. Order your results so the winners are shown first, then ordered by constituency.**

In [6]:
ge %>%
    filter(constituency>='S14000021' & 
           constituency<='S14000026' &
           yr==2017) %>%
    group_by(constituency) %>% 
    mutate(posn=row_number(desc(votes))) %>%
    ungroup %>% 
    select(constituency, party, votes, posn) %>%
    arrange(posn, constituency)

constituency,party,votes,posn
<chr>,<chr>,<int>,<int>
S14000021,Conservative,21496,1
S14000022,SNP,18509,1
S14000023,SNP,19243,1
S14000024,Labour,26269,1
S14000025,SNP,17575,1
S14000026,Liberal Democrats,18108,1
S14000021,Labour,14346,2
S14000022,Labour,15084,2
S14000023,Labour,17618,2


## 5. Winners Only

You can use [SELECT within SELECT](https://sqlzoo.net/wiki/SELECT_within_SELECT_Tutorial) to pick out only the winners in Edinburgh.

**Show the parties that won for each Edinburgh constituency in 2017.**

In [7]:
ge %>%
    filter(constituency>='S14000021' & 
           constituency<='S14000026' &
           yr==2017) %>%
    group_by(constituency) %>%
    mutate(posn=row_number(desc(votes))) %>%
    ungroup %>% 
    filter(posn==1) %>%
    select(constituency, party) %>%
    arrange(constituency)

constituency,party
<chr>,<chr>
S14000021,Conservative
S14000022,SNP
S14000023,SNP
S14000024,Labour
S14000025,SNP
S14000026,Liberal Democrats


## 6. Scottish seats

You can use **COUNT** and **GROUP BY** to see how each party did in Scotland. Scottish constituencies start with 'S'

**Show how many seats for each party in Scotland in 2017.**

In [8]:
ge %>%
    filter(str_starts(constituency, 'S') & yr==2017) %>%
    group_by(constituency) %>% 
    mutate(posn=row_number(desc(votes))) %>%
    ungroup %>%
    filter(posn==1) %>% 
    group_by(party) %>%
    summarise(seats=n()) %>%
    arrange(party)

party,seats
<chr>,<int>
Conservative,12
Labour,9
Liberal Democrats,4
SNP,34


In [9]:
dbDisconnect(con)